<a href="https://colab.research.google.com/github/ParvG2005/Parv/blob/main/QuantumNeuralNetworkIrisDataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane scikit-learn numpy



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 573.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 59.3 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, np.pi))
X = scaler.fit_transform(X)

encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

n_qubits = 4
n_layers = 2
dev = qml.device("default.qubit", wires=n_qubits)

def circuit(weights, x):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)
    for layer in range(n_layers):
        for i in range(n_qubits):
            qml.Rot(*weights[layer, i], wires=i)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        if n_qubits > 1:
            qml.CNOT(wires=[n_qubits - 1, 0])
    return [qml.expval(qml.PauliZ(i)) for i in range(3)]

qnode = qml.QNode(circuit, dev, interface="autograd")

def qnn(weights, x):
    return qnode(weights, x=x)

def cost(weights, X, Y):
    predictions = np.array([qnn(weights, x) for x in X])
    softmax = np.exp(predictions) / np.sum(np.exp(predictions), axis=1, keepdims=True)
    loss = -np.sum(Y * np.log(softmax + 1e-10)) / len(X)
    return loss

def train(X_train, y_train, epochs=50, learning_rate=0.1):
    weights = np.random.uniform(0, 2 * np.pi, (n_layers, n_qubits, 3))
    weights.requires_grad = True
    opt = qml.AdamOptimizer(learning_rate)
    for epoch in range(epochs):
        weights, current_cost = opt.step_and_cost(lambda w: cost(w, X_train, y_train), weights)
        if epoch % 5 == 0:
            print(f"Epoch {epoch}: Cost = {current_cost:.4f}")
    return weights

try:
    trained_weights = train(X_train, y_train, epochs=50)
except Exception as e:
    print(f"Error during training: {str(e)}")

def evaluate(weights, X, Y):
    correct = sum(
        np.argmax(qnn(weights, x)) == np.argmax(true)
        for x, true in zip(X, Y)
    )
    return correct / len(X)

if 'trained_weights' in locals():
    test_accuracy = evaluate(trained_weights, X_test, y_test)
    print(f"\nTest Accuracy: {test_accuracy:.2%}")
else:
    print("Training failed, cannot evaluate.")


Epoch 0: Cost = 1.1408
Epoch 5: Cost = 0.9722
Epoch 10: Cost = 0.8693
Epoch 15: Cost = 0.7894
Epoch 20: Cost = 0.7663
Epoch 25: Cost = 0.7358
Epoch 30: Cost = 0.6935
Epoch 35: Cost = 0.6768
Epoch 40: Cost = 0.6680
Epoch 45: Cost = 0.6591

Test Accuracy: 93.33%
